In [172]:
import os
from twilio.rest import Client

# Definimos las configuraciones de CoinAPI y su API
from CoinAPI_key_config import API_KEY

# Definimos las configuraciones para el Twilio API - Mensaje wsp
from twilio_wsp_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER

# Librería para la hora actual
import time

# Para preparar y enviar solicitudes HHTP
from requests import Request, Session
# Para el manejo de errores
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
# Para la codificación y decodificación de JSON
import json

# Para el análisis de datos 
import pandas as pd
# Para enviar solitudes HTTP
import requests

# Para agregar barras de progreso
from tqdm import tqdm

# Para obtener la fecha actual
from datetime import datetime

### Definir la URL

In [3]:
# URL de la API

coin_api_key = API_KEY

url_crip = 'https://rest.coinapi.io/v1/assets/apikey-'+ coin_api_key
url_crip

'https://rest.coinapi.io/v1/assets/apikey-C53C9FB7-F7A0-49FC-A0FA-E4781F11E8F8'

### Realiza Petición

In [26]:
# Realizar la petición 
response_crip= requests.get(url_crip).json()
response_crip

[{'asset_id': 'USD',
  'name': 'US Dollar',
  'type_is_crypto': 0,
  'data_quote_start': '2014-02-24T17:43:05.0000000Z',
  'data_quote_end': '2023-07-05T00:00:00.0000000Z',
  'data_orderbook_start': '2014-02-24T17:43:05.0000000Z',
  'data_orderbook_end': '2023-07-05T00:00:00.0000000Z',
  'data_trade_start': '2010-07-17T23:09:17.0000000Z',
  'data_trade_end': '2023-07-05T00:00:00.0000000Z',
  'data_symbols_count': 191397,
  'volume_1hrs_usd': 13488524302840.94,
  'volume_1day_usd': 1069934451920286.4,
  'volume_1mth_usd': 6.930284255873227e+16,
  'id_icon': '0a4185f2-1a03-4a7c-b866-ba7076d8c73b',
  'data_start': '2010-07-17',
  'data_end': '2023-07-05'},
 {'asset_id': 'BTC',
  'name': 'Bitcoin',
  'type_is_crypto': 1,
  'data_quote_start': '2014-02-24T17:43:05.0000000Z',
  'data_quote_end': '2023-07-05T00:00:00.0000000Z',
  'data_orderbook_start': '2014-02-24T17:43:05.0000000Z',
  'data_orderbook_end': '2023-07-05T00:00:00.0000000Z',
  'data_trade_start': '2010-07-17T23:09:17.0000000Z',

### Interactura con el JSON

In [44]:
# Obtener el Nombre de la Criptomoneda
response_crip[1]['name']

'Bitcoin'

In [77]:
# Obtener el valor de la Criptomoneda
round(response_crip[1]['price_usd'],0)

30080.0

### Crear Dataframe

In [158]:
# Función que recorre y extrae los valores de las Criptomendas
def get_crip(response, i):
    nombre = response_crip[i]['name']
    precio = response_crip[i].get('price_usd')
    if precio is not None:
        precio = round(precio, 0)
    return nombre, precio

In [159]:
# Recorrer los valores de las criptomonedas
datos = []
for i in tqdm(range(len(response_crip)), colour='blue'):
    datos.append(get_crip(response_crip, i))

datos

100%|████████████████████████████████████████████████████████████████████████| 17938/17938 [00:00<00:00, 943096.70it/s]


[('US Dollar', None),
 ('Bitcoin', 30080.0),
 ('Zloty', 0.0),
 ('Euro', 1.0),
 ('Yuan Renminbi', 0.0),
 ('Yen', 0.0),
 ('Australian Dollar', 1.0),
 ('Swiss Franc', 1.0),
 ('Swedish Krona', 0.0),
 ('Pound Sterling', 1.0),
 ('Danish Krone', 0.0),
 ('Russian Ruble', 0.0),
 ('Singapore Dollar', 1.0),
 ('Canadian Dollar', 1.0),
 ('New Zealand Dollar', 1.0),
 ('Baht', 0.0),
 ('NIS', 0.0),
 ('Litecoin', 97.0),
 ('Won', 0.0),
 ('VeChain (pre-swap)', 0.0),
 ('Ripple', 0.0),
 ('Namecoin', 2.0),
 ('Tether', 1.0),
 ('DogeCoin', 0.0),
 ('Novacoin', None),
 ('Feathercoin', 0.0),
 ('Peercoin', 0.0),
 ('Terracoin', 0.0),
 ('El Salvador Colon', 0.0),
 ('ORO', None),
 ('Torcoin', None),
 ('StarCoin', 0.0),
 ('BitShares', 0.0),
 ('Nxt', 0.0),
 ('Blakecoin', None),
 ('Pakistan Rupee', 0.0),
 ('WINCOIN', None),
 ('Leone', None),
 ('DARK', 0.0),
 ('UCoin', 0.0),
 ('Brazilian Real', 0.0),
 ('Chilean Peso', 0.0),
 ('MonetaryUnit', None),
 ('SongCoin', None),
 ('PayCon', 0.0),
 ('NAVCoin', 0.0),
 ('RICE', None

In [160]:
# Definimos los nombre de las columnas de DF
columns = ['Nombre', 'Precio$']

# Crear el DF
df_crip = pd.DataFrame(datos,columns=columns)
df_crip = df_crip.sort_values(by = 'Precio$',ascending = False)
df_crip

,Nombre,Precio$
2233,BitTube,1.694628e+12
3147,RTO,1.503660e+12
3840,NEOX,9.900086e+11
378,BitCoal,6.094156e+11
3075,VEIL,2.467447e+11
...,...,...
17933,AUDI,NaN
17934,BINGO,NaN
17935,MGPV2,NaN
17936,QNN,NaN


### Data Cleaning

In [161]:
# Realizar copia del DF
df_crip = df_crip.copy()

In [162]:
# Eliminar valores NULL en "Precio$"
df_crip = df_crip.dropna(subset=['Precio$'])
df_crip.count()

Nombre     4622
Precio$    4622
dtype: int64

In [163]:
# Convertir a tipo de datos INT a "Precio$"
df_crip['Precio$']=df_crip['Precio$'].astype('int')

In [171]:
# Filtramos para valores > 0 y < 31000
df_crip = df_crip.loc[(df_crip['Precio$'] > 0) & (df_crip['Precio$'] <= 31000)]

In [170]:
# Obtenemos el TOP 5
df_cript_top=df_crip.head(5)
df_cript_top

,Nombre,Precio$
2617,BTCB,30131
686,WBTC,30117
2660,RBTC,30113
4505,TBTC,30098
3213,WBT,30091


### Template de Mensaje de Wsp

In [181]:
'\nHola! \n\n Los valores de criptomonedas en $ son los siguiente: \n\n'+ str(df_cript_top)

'\nHola! \n\n Los valores de criptomonedas en $ son los siguiente: \n\n     Nombre  Precio$\n2617   BTCB    30131\n686    WBTC    30117\n2660   RBTC    30113\n4505   TBTC    30098\n3213    WBT    30091'

In [173]:
PHONE_NUMBER

'+14155238886'

### Mensaje por WSP desde Twilio

In [184]:
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='whatsapp:' + PHONE_NUMBER,
  body='\nHola! \n\n Los valores de criptomonedas en $ son los siguiente: \n\n'+ str(df_cript_top.to_string(index=False)),
  to='whatsapp:+51963290272'
)

print(message.sid)

SM91344dae92772e807d338b46f1149b75
